<a href="https://colab.research.google.com/github/rj-pal/CEFR-text-classification/blob/main/2_data_modeling_keras_one_stop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Mar  2 14:06:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# !cp /content/drive/My\ Drive/data/English/preprocessing_nlp.py

In [ ]:
import sys
sys.path.append('drive/MyDrive/data/English')

In [ ]:
# !pip install -r requirements.txt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

import preprocessing_nlp as pp
# import cleaning_nlp as cc

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
url_train = 'https://raw.githubusercontent.com/rj-pal/CEFR-text-classification/main/data/one_stop_train.csv'
url_test = 'https://raw.githubusercontent.com/rj-pal/CEFR-text-classification/main/data/one_stop_test.csv'
df_train = pd.read_csv(url_train)
df_test = pd.read_csv(url_test)


In [ ]:
df_train

,index,documents,doc_list,avg_num_words,total_num_sents,total_num_words,words_per_sents,level,doc_id
0,0,SeaWorld's profits fell by 84% and customers a...,['SeaWorld’s profits fell by 84% and customers...,25.00,1,25,[25],Ele,0
1,1,The company teaches dolphins and killer whales...,['The company teaches dolphins and killer whal...,17.00,1,17,[17],Ele,0
2,2,It says fewer people are going to its parks an...,['It says fewer people are going to its parks ...,13.00,1,13,[13],Ele,0
3,3,SeaWorld has been in the news since the 2013 d...,['SeaWorld has been in the news since the 2013...,33.00,1,33,[33],Ele,0
4,4,Animal rights organizations say that orcas kep...,['Animal rights organizations say that orcas k...,14.00,2,28,"[17, 11]",Ele,0
...,...,...,...,...,...,...,...,...,...
6545,7390,One major problem is complicity between senior...,['One major problem is complicity between seni...,26.00,3,78,"[15, 32, 31]",Adv,566
6546,7391,As the number of tigers has increased over the...,['As the number of tigers has increased over t...,19.25,4,77,"[17, 21, 14, 25]",Adv,566
6547,7392,Wildlife authorities captured a tiger in Chitw...,['Wildlife authorities captured a tiger in Chi...,19.00,4,76,"[23, 12, 14, 27]",Adv,566
6548,7393,"Diwakar Chapagain, who heads a World Wildlife ...","['Diwakar Chapagain, who heads a World Wildlif...",30.00,1,30,[30],Adv,566


In [ ]:
d_train = df_train[['documents', 'level']]
d_test = df_test[['documents', 'level']]

In [ ]:
d_train.shape, d_test.shape

((6550, 2), (845, 2))

In [ ]:
sentence_data_train = pp.get_sentences_dataframe(d_train, 'documents', 'level')
sentence_data_test = pp.get_sentences_dataframe(d_test, 'documents', 'level')

In [ ]:
df_processed_train = pp.preprocess_all(sentence_data_train, 'documents', stop_words=True)
df_processed_test = pp.preprocess_all(sentence_data_test, 'documents', stop_words=True)

In [ ]:
df_processed_train

,documents,num_of_words,level,doc_id,documents_clean,Lemmatized,Stemmed
0,SeaWorld's profits fell by 84% and customers a...,25,Ele,0,seaworld profits fell customers staying away w...,seaworld profit fell customer staying away wat...,seaworld profit fell custom stay away water th...
1,The company teaches dolphins and killer whales...,17,Ele,1,company teaches dolphins killer whales tricks ...,company teach dolphin killer whale trick front...,compani teach dolphin killer whale trick front...
2,It says fewer people are going to its parks an...,13,Ele,2,says fewer people going parks profits reduced,say fewer people going park profit reduced,say fewer peopl go park profit reduc
3,SeaWorld has been in the news since the 2013 d...,33,Ele,3,seaworld news since thedocumentary film blackf...,seaworld news since thedocumentary film blackf...,seaworld news sinc thedocumentari film blackfi...
4,Animal rights organizations say that orcas kep...,17,Ele,4,animal rights organizations say orcas kept tan...,animal right organization say orca kept tank d...,anim right organ say orca kept tank die younge...
...,...,...,...,...,...,...,...
17718,Chitwan National Park in central Nepal has the...,27,Adv,6547,chitwan national park central nepal highest nu...,chitwan national park central nepal highest nu...,chitwan nation park central nepal highest numb...
17719,"Diwakar Chapagain, who heads a World Wildlife ...",30,Adv,6548,diwakar chapagain heads world wildlife fund ne...,diwakar chapagain head world wildlife fund nep...,diwakar chapagain head world wildlif fund nepa...
17720,"In Nepal, kings used to stand on tiger skins i...",17,Adv,6549,nepal kings used stand tiger skins front stuff...,nepal king used stand tiger skin front stuffed...,nepal king use stand tiger skin front stuf tig...
17721,Some affluent Nepali have mounted tiger heads ...,14,Adv,6549,affluent nepali mounted tiger heads walls livi...,affluent nepali mounted tiger head wall living...,affluent nepali mount tiger head wall live room


In [ ]:
data_train = df_processed_train[['Lemmatized', 'level']]
data_test = df_processed_test[['Lemmatized', 'level']]

In [ ]:
r = max(data_train.Lemmatized.values.tolist())
len(r)

88

In [ ]:
s = [sen for sen in data_train.Lemmatized.values.tolist()]


In [ ]:
ws = [words for sen in data_train.Lemmatized.values.tolist() for words in sen.split()]
wl = [len(word) for word in ws]
max(wl)

40

In [ ]:
# df
sl = [len(sen.split()) for sen in data_train.Lemmatized.values.tolist()]
sum(s for s in sl)/len(sl)

10.366077977769

In [ ]:
# sorted(sl)[::-1]

In [ ]:
w = [word for words in data_train.Lemmatized.values.tolist() for word in words.split() ]

In [ ]:
len(set(w))

14015

In [ ]:
y_integers = []
for level in data_train.level:
  if level == 'Ele':
    y_integers.append(0)
  elif level == 'Int':
    y_integers.append(1)
  else:
    y_integers.append(2)
data_train['level'] = y_integers

y_integers = []
for level in data_test.level:
  if level == 'Ele':
    y_integers.append(0)
  elif level == 'Int':
    y_integers.append(1)
  else:
    y_integers.append(2)
data_test['level'] = y_integers

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
data_train.groupby('level').count()

,Lemmatized
level,
0,5183
1,6109
2,6431


In [ ]:
import tensorflow as tf
from tensorflow.python import keras
from tensorflow.keras import preprocessing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Dropout, SpatialDropout1D, LSTM, Reshape, Embedding, TextVectorization
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.model_selection import GridSearchCV


In [ ]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 8000
# Max number of words in each sentence.
MAX_SEQUENCE_LENGTH = 40
# This is fixed for LSTM layer.
# EMBEDDING_DIM = 15
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(data_train['Lemmatized'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 14014 unique tokens.


In [ ]:
X = tokenizer.texts_to_sequences(data_train['Lemmatized'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (17723, 40)


In [ ]:
X_test = tokenizer.texts_to_sequences(data_test['Lemmatized'].values)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)


In [ ]:
y = to_categorical(data_train['level'].values)
y_test = to_categorical(data_test['level'].values)


In [ ]:
def create_model(dropout_rate=0.1, EMBEDDING_DIM=5):
  model = Sequential()
  model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
  model.add(SpatialDropout1D(dropout_rate))
  model.add(LSTM(EMBEDDING_DIM, dropout=dropout_rate, recurrent_dropout=dropout_rate))
  model.add(Dense(3, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [ ]:
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=32, verbose=0)
# define the grid search parameters
EMBEDDING_DIM = [5, 20, 65,105]
dropout_rate = [0.2, 0.5, 0.9]
param_grid = dict(dropout_rate=dropout_rate, EMBEDDING_DIM=EMBEDDING_DIM)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(X, y)
# summarize results
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  """Entry point for launching an IPython kernel.


In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.060146 using {'EMBEDDING_DIM': 105, 'dropout_rate': 0.2}
0.025107 (0.019972) with: {'EMBEDDING_DIM': 5, 'dropout_rate': 0.2}
0.017885 (0.013195) with: {'EMBEDDING_DIM': 5, 'dropout_rate': 0.5}
0.029564 (0.041810) with: {'EMBEDDING_DIM': 5, 'dropout_rate': 0.9}
0.045983 (0.042341) with: {'EMBEDDING_DIM': 20, 'dropout_rate': 0.2}
0.035432 (0.033566) with: {'EMBEDDING_DIM': 20, 'dropout_rate': 0.5}
0.020594 (0.029124) with: {'EMBEDDING_DIM': 20, 'dropout_rate': 0.9}
0.053544 (0.045771) with: {'EMBEDDING_DIM': 65, 'dropout_rate': 0.2}
0.042993 (0.038440) with: {'EMBEDDING_DIM': 65, 'dropout_rate': 0.5}
0.011171 (0.013142) with: {'EMBEDDING_DIM': 65, 'dropout_rate': 0.9}
0.060146 (0.039826) with: {'EMBEDDING_DIM': 105, 'dropout_rate': 0.2}
0.046547 (0.041841) with: {'EMBEDDING_DIM': 105, 'dropout_rate': 0.5}
0.012300 (0.010020) with: {'EMBEDDING_DIM': 105, 'dropout_rate': 0.9}
